Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Damir"
COLLABORATORS = ""

---

# Adding constraints derived from a structure

Before beginning, please copy the files out of the Session09 directory and into the `pyrbc_202103_notebooks/inputs` directory. Ignore this step if you ran the `adding_existing_constraints` notebook first and you've already copied these files over.

In [ ]:
# You have to rerun this cell each time you start a new notebook or do a "factory reset".
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.mount_pyrosetta_install() #Instead of pyrosettacolabsetup.setup
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

Drive already mounted at /content/google_drive; to attempt to forcibly remount, call drive.mount("/content/google_drive", force_remount=True).
Notebook is set for PyRosetta use in Colab.  Have fun!


Now cd into the correct directory

In [ ]:
%cd google_drive/MyDrive/temp_pyrbc_202103_notebooks

/content/google_drive/MyDrive/temp_pyrbc_202103_notebooks


We initialize PyRosetta without any command line options.

In [ ]:
from pyrosetta import *
init()

PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2021.21+release.882e5c1ab85c8c251fce4eb3e1e0504af590786a 2021-05-26T14:40:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r284 2021.21+release.882e5c1ab85 882e5c1ab85c8c251fce4eb3e1e0504af590786a http://www.pyrosetta.org 2021-05-26T14:40:53
core.init: command: PyRosetta -ex1 -ex2aro -database /content/google_drive/MyDrive/prefix/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1746069396 seed_offset=0 real_seed=1746069396
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1746069396 RG_type=mt19937


Initiliazing pose and score function.

In [ ]:
pose = pose_from_pdb("inputs/1ubq.pdb")
sfxn = get_fa_scorefxn()
sfxn(pose)

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 984 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.72808 seconds.
core.import_pose.import_pose: File 'inputs/1ubq.pdb' automatically determined to be of type PDB
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/H

32.67775371933962

Let's prime the score function with a constraint score term. Let's also print out the score before we've added any constraints to the pose so we can refer back to it later.

In [ ]:
from pyrosetta.rosetta.core.scoring import *
sfxn.set_weight(atom_pair_constraint, 1.0)
sfxn.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.647    -397.647
 fa_rep                       0.550     103.707      57.039
 fa_sol                       1.000     242.952     242.952
 fa_intra_rep                 0.005     355.469       1.777
 fa_intra_sol_xover4          1.000      16.826      16.826
 lk_ball_wtd                  1.000      -8.756      -8.756
 fa_elec                      1.000    -113.091    -113.091
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -18.828     -18.828
 hbond_lr_bb                  1.000     -23.132     -23.132
 hbond_bb_sc                  1.000      -7.389      -7.389
 hbond_sc                     1.000      -1.549      -1.549
 dslf_fa13                    1.250       0.000       0.000
 atom_pa

To define where the constraints need to be applied, we will need to use residue selectors. Let's use a `LayerSelector` to select the surface residues.

In [ ]:
from pyrosetta.rosetta.core.select.residue_selector import *
surface_res = LayerSelector()
surface_res.set_layers(False, False, True) # use "?" to look at this function arguments

core.select.residue_selector.LayerSelector: Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: Setting core=false boundary=false surface=true in LayerSelector.


To check which residues are selected on the pose, we can apply it. `1` means that the residue has been selected (and is a surface residue), `0` means that it has not.

In [ ]:
surface_res.apply(pose)

vector1_bool[0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]

Now let's create atom pair constraints. For this, we use `AtomPairConstraintGenerator`. **By default, it uses the Sum Of Gaussians function to score the distances between atoms.** The "sum" part is not really used, because there is just one gaussian. This gaussian turns a distance into a probability, `prob`, and the energy is taken as the `-ln(prob)`.   We will pass the residue selector to the generator, set the maximum distance at under which it will create atom pair constraints (5.0 Ang), the standard deviation of the gaussians, and set the constraints to only be generated between pairs of C-alpha atoms.

In [ ]:
from pyrosetta.rosetta.protocols.constraint_generator import *
apcg = AtomPairConstraintGenerator()
apcg.set_residue_selector(surface_res)
apcg.set_max_distance(5.0)
apcg.set_sd(1.0)
apcg.set_ca_only(True)

While we could directly apply the `AtomPairConstraintGenerator` to the pose, the recommended method is to add it to an `AddConstraints` object. Multiple constraint generators can be added to this. We then apply all the constraints.

In [ ]:
add_csts = AddConstraints()
add_csts.add_generator(apcg)
add_csts.apply(pose)

protocols.constraint_generator.AddConstraints: Adding 9 constraints generated by ConstraintGenerator named unnamed_constraint_generator


Let's check if the constraints were applied.

In [ ]:
sfxn.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.647    -397.647
 fa_rep                       0.550     103.707      57.039
 fa_sol                       1.000     242.952     242.952
 fa_intra_rep                 0.005     355.469       1.777
 fa_intra_sol_xover4          1.000      16.826      16.826
 lk_ball_wtd                  1.000      -8.756      -8.756
 fa_elec                      1.000    -113.091    -113.091
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -18.828     -18.828
 hbond_lr_bb                  1.000     -23.132     -23.132
 hbond_bb_sc                  1.000      -7.389      -7.389
 hbond_sc                     1.000      -1.549      -1.549
 dslf_fa13                    1.250       0.000       0.000
 atom_pa

The sum of gaussians function acts more like a reward. A large neagtive number indicates that all 9 constraints were satisfied. What happens when we disrupt some surface interactions in a perturbed structure?

In [ ]:
pose_perturbed = pose.clone()  # copy the conformation and the constraints
pose_perturbed.set_phi(25, -130)
pose_perturbed.set_psi(25,  145)

In [ ]:
sfxn.show(pose_perturbed)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -575.051    -575.051
 fa_rep                       0.550   55367.773   30452.275
 fa_sol                       1.000     413.031     413.031
 fa_intra_rep                 0.005     355.301       1.777
 fa_intra_sol_xover4          1.000      16.485      16.485
 lk_ball_wtd                  1.000     -16.198     -16.198
 fa_elec                      1.000     -84.545     -84.545
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -15.276     -15.276
 hbond_lr_bb                  1.000     -13.621     -13.621
 hbond_bb_sc                  1.000      -7.087      -7.087
 hbond_sc                     1.000      -2.165      -2.165
 dslf_fa13                    1.250       0.000       0.000
 atom_pa

The reward is much smaller owing to the perturbation of the pose.